In [5]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import emcee
import paths
import multiprocessing as mp

import warnings
warnings.filterwarnings('ignore')
from orbitize import driver, read_input, results

# FIX THE SEED SO THAT RESULTS ARE REPRODUCIBLE

np.random.seed(44)

# load in astrometry of V773 Tau
orb_file = paths.data / "apj416631t1_ascii_mod_2xerr.txt"

# MCMC parameters
num_temps = 20
num_walkers = 1000
num_threads = mp.cpu_count()
print("number of processors:{}".format(num_threads))

plx    = 7.70  # from Torres 2012 page 7 - better than GAIA as explicit solve using VLBI data
plxerr = 0.19  # from Torres 2012 page 7

myDriver = driver.Driver(
    orb_file, # data file
    'MCMC',        # choose from: ['OFTI', 'MCMC']
    1,             # number of planets in system
    5.27,          # total mass [M_sun] Boden 2012 Table 4
    plx,         # system parallax [mas]
    mass_err=0.65, # mass error [M_sun] Boden 2012 Table 4
    plx_err=plxerr, # parallax error [mas]
    mcmc_kwargs={'num_temps': num_temps, 'num_walkers': num_walkers, 'num_threads': num_threads}
)

# full run parameters
total_orbits = 1000000 # number of steps x number of walkers (at lowest temperature)
burn_steps = 2000 # steps to burn in per walker
thin = 4

# for testing
#total_orbits = 10000 # number of steps x number of walkers (at lowest temperature)
#burn_steps = 0 # steps to burn in per walker

orbits = myDriver.sampler.run_sampler(total_orbits, burn_steps=burn_steps, thin=thin)

hdf5_filename = paths.data / 'v773_tau_AB.hdf5'

import os

# To avoid weird behaviours, delete saved file if it already exists from a previous run of this notebook
if os.path.isfile(hdf5_filename):
    print('deleting old bundle {}'.format(hdf5_filename))
    os.remove(hdf5_filename)
    
myResults = myDriver.sampler.results
myResults.save_results(paths.data / 'v773_tau_AB.hdf5')

number of processors:16
Starting Burn in

Burn in complete. Sampling posterior now.


/Users/kenworthy/opt/anaconda3/lib/python3.9/site-packages/orbitize/priors.py:463: RuntimeWarning: invalid value encountered in log
  lnprob = np.log(np.sin(element_array)/normalization)
/Users/kenworthy/opt/anaconda3/lib/python3.9/site-packages/orbitize/priors.py:354: RuntimeWarning: invalid value encountered in log
  lnprob = -np.log((element_array*normalizer))
/Users/kenworthy/opt/anaconda3/lib/python3.9/site-packages/orbitize/priors.py:354: RuntimeWarning: invalid value encountered in log
  lnprob = -np.log((element_array*normalizer))
/Users/kenworthy/opt/anaconda3/lib/python3.9/site-packages/orbitize/priors.py:354: RuntimeWarning: invalid value encountered in log
  lnprob = -np.log((element_array*normalizer))
/Users/kenworthy/opt/anaconda3/lib/python3.9/site-packages/orbitize/priors.py:463: RuntimeWarning: invalid value encountered in log
  lnprob = np.log(np.sin(element_array)/normalization)
/Users/kenworthy/opt/anaconda3/lib/python3.9/site-packages/orbitize/priors.py:463: Runtim

10/10 steps completed
Run complete
